In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
#import shutil
#import os
import json
import sources.REmodeling as mod

C:\Users\Severus\AppData\Roaming\Python\Python39\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\Severus\AppData\Roaming\Python\Python39\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not fi

In [3]:
class CONFIG: 
    EPS = 1e-8 
    EPOCHS = 3 # 3~5
    BATCH_SIZE = 16 # 8, 32
    LEARNING_RATE = 3e-5 # 1e-5
    MAX_LENGTH = 128 # 256
    ENCODER_TYPE = 'bert-base' # large, RoBERTa, DeBERTa
    OPTIMIZER ='adamw'   
    DEVICE_TYPE = "cuda" # Cuda or alternative
    SAVE_MODEL=False
    TASK='RE-classification'
config=CONFIG()

In [4]:

class Datapack:
    def __init__(this,train,test,val):
        this.test_ds,this.reladict,this.testx,this.testy=this.load_dataset(test)
        this.valid_ds,reladict,this.validx,this.validy=this.load_dataset(val,this.reladict,shuffle=True)
        this.train_ds,this.reladict,this.trainx,this.trainy=this.load_dataset(train,this.reladict,shuffle=True)
        this.invdict=np.zeros(len(this.reladict)).tolist()
        for k,v in reladict.items():
            this.invdict[v]=k
        
    def load_dataset(this,path,reladict={},shuffle=False):
        with open(path,'r',encoding='UTF-8') as f:
            data=[json.loads(line) for line in f]
        sent=[]
        relation=[]
        for d in data:
            token=d['token']
            for i in range(d['h']['pos'][0],d['h']['pos'][1]):
                token[i]='[MASK]'
            for i in range(d['t']['pos'][0],d['t']['pos'][1]):
                token[i]='[MASK]'
            sent.append([' '.join(token)])
            if reladict.get(d['relation'],False)==False:
                reladict.setdefault(d['relation'],len(reladict))
            relation.append(reladict[d['relation']])
        temp=np.zeros(shape=(len(relation),len(reladict)))
        #print(temp.shape)
        #print(reladict)
        for i in range(len(relation)):
            temp[i][relation[i]]=1
        if shuffle:
            data=tf.data.Dataset.from_tensor_slices((sent,temp)).shuffle(5000).batch(CONFIG.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        else:
            data=tf.data.Dataset.from_tensor_slices((sent,temp)).batch(CONFIG.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        return data,reladict,sent,relation


In [5]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
datapack=Datapack('semeval_train.txt','semeval_train.txt','semeval_train.txt')
#for text_batch, label_batch in train_ds.take(1):
  #for i in range(3):
    #print(f'Review: {text_batch.numpy()[i]}')
    #label = label_batch.numpy()[i]
    #print(f'Label : {label}')
    #print(len(label))

In [6]:
if True:
    with open('results/NRE_test_result.json','w',encoding='UTF-8')as f:
        json.dump({},f,indent=2)

In [7]:
result,model=mod.modeling(config,datapack)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 Preprocessor/Tokenizer (KerasL  {'input_word_ids':   0          ['text[0][0]']                   
 ayer)                          (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

NameError: name 'testx' is not defined